# Hash Map from scratch in Python
- We’ll use chaining (linked list or just a Python list) to handle collisions, because it’s the easiest to understand.

## 🔹 Step 1: Basic Idea
- A hash table is basically an array (list).
- We use a hash function to decide which index a key-value pair should go to.
- If two keys map to the same index → store them in a bucket (list at that index).

In [1]:
class HashTable:
  def __init__(self, size=10):
    # create an array (list) of empty lists (buckets)
    self.size = size
    self.table = [[] for _ in range(size)]

  def _hash(self, key):
    """Hash function: convert key into an index"""
    return hash(key) % self.size

  def insert(self, key, value):
    """Insert key-value pair"""
    index = self._hash(key)
    # check if key already exists, update value
    for pair in self.table[index]:
      if pair[0] == key:
        pair[1] = value
        return
    # otherwise, return new pair
    self.table[index].append([key, value])

  def get(self, key):
    """Retrive value by key"""
    index = self._hash(key)
    for pair in self.table[index]:
      if pair[0] == key:
        return pair[1]
    return None
  
  def delete(self, key):
    """Delete key-value pair"""
    index = self._hash(key)
    for i, pair in enumerate(self.table[index]):
      if pair[0] == key:
        del self.table[index][i]
        return True
    return False

  def display(self):
    """Show the hash table"""
    for i, bucket in enumerate(self.table):
      print(f"{i}: {bucket}")

ht = HashTable()

# Insert values
ht.insert("Alice", 123)
ht.insert("Bob", 456)
ht.insert("Charlie", 789)

# Display table
ht.display()

# Access Values
print("Bob's number: ", ht.get("Bob"))
print("Ashish's number: ", ht.get("Ashish"))

# Update value
ht.insert("Alice", 999)
print("Alice's number: ", ht.get("Alice"))

# Delete a key
ht.delete("Charlie")

ht.display()

0: [['Charlie', 789]]
1: [['Alice', 123]]
2: []
3: []
4: []
5: []
6: []
7: [['Bob', 456]]
8: []
9: []
Bob's number:  456
Ashish's number:  None
Alice's number:  999
0: []
1: [['Alice', 999]]
2: []
3: []
4: []
5: []
6: []
7: [['Bob', 456]]
8: []
9: []


### Class header and constructor

```
class HashTable:
  def __init__(self, size=10):
    # create an array (list) of empty lists (buckets)
    self.size = size
    self.table = [[] for _ in range(size)]
```

- `class HashTable:` — defines a new class named HashTable.
- `def __init__(self, size=10):` — constructor; size is the initial number of buckets (defaults to 10).
- Comment: # create an array (list) of empty lists (buckets) — clarifies intent: each slot is a bucket.
- `self.size = size` — store number of buckets so other methods can use it.
- `self.table = [[] for _ in range(size)]` — create a list containing size empty lists. Each of these inner lists is a bucket that will hold [key, value] pairs. Using a list comprehension ensures each bucket is a distinct list.

Note: Because buckets are lists, collisions are handled by appending pairs to the bucket list.

### Hash Function

```
def _hash(self, key):
    """Hash function: convert key into an index"""
    return hash(key) % self.size
```

- `def _hash(self, key)`: — internal helper method to compute an index from a key.
- `return hash(key) % self.size` — `hash(key)` returns a Python integer for the key (works for any hashable key like strings, ints, tuples). The modulo (% self.size) maps the integer into the valid bucket range 0..size-1.

### Important notes about `hash()`:
- Keys must be hashable (immutable types like `int`, `str`, `tuple` are ok; `list` is not).
- Python applies hash randomization for some types (strings) across interpreter runs, so indices may vary between runs — not an issue functionally.
- Collisions (different keys → same index) are expected and handled by the buckets.

### Insert Method

```
def insert(self, key, value):
  index = self._hash(key)
  for pair in self.table[index]:
    if pair[0] == key:
      pair[1] = value
      return
  self.table[index].append([key, value])
```

- `index = self._hash(key)` — compute bucket index for the key.
- for pair in `self.table[index]`: — iterate through existing (key, value) pairs in that bucket.
- if `pair[0] == key`: — if the key already exists in the bucket (we compare by equality).
- `pair[1] = value` — update the existing pair’s value. (Pairs stored as lists [key, value] so we can update in place.)
- `return` — exit after update (don't add a duplicate).
- `self.table[index].append([key, value])` — if key wasn’t present, append a new pair list to the bucket.

### Design choices / considerations:
- Storing pairs as lists ([key, value]) makes updating pair[1] easy. If you use tuples you must replace the entire tuple.
- This method does not resize the table. If you insert many items, buckets will grow and operation worst-case becomes O(n).

### Get Method

```
def get(self, key):
  index = self._hash(key)
  for pair in self.table[index]:
    if pair[0] == key:
      return pair[1]
  return None
```

- `index = self._hash(key)` — get bucket index.
- for pair in `self.table[index]`: — search bucket for the key.
- `if pair[0] == key: return pair[1]` — return the value if found.
- `return None` — if not found, return None.

### Caveat: returning None for "not found" can be ambiguous if a stored value is actually None. Alternatives:
- Raise `KeyError` (like Python `dict.__getitem__`).
- Provide `get(key, default=None)` API so user can pass a default.

### Delete Method

```
def delete(self, key):
  index = self._hash(key)
  for i, pair in enumerate(self.table[index]):
    if pair[0] == key:
      del self.table[index][i]
      return True
  return False
```

- `index = self._hash(key)` — compute the bucket.
- `for i, pair in enumerate(self.table[index])`: — iterate with index i.
- `if pair[0] == key`: — found the key.
- `del self.table[index][i]` — remove the pair from the bucket.
- `return True` — signal successful deletion.
- `return False` — if not found, return False.

Complexity: deletion in a bucket requires shifting elements in the Python list (O(k) where k is bucket length), but average-case still O(1) for a well-sized table.

### Display method

```
def display(self):
  for i, bucket in enumerate(self.table):
    print(f"{i}: {bucket}")
```

- `for i, bucket in enumerate(self.table)`: — iterate over index and bucket.
- `print(f"{i}: {bucket}")` — print bucket index and the list of pairs in that bucket — helpful for debugging/visualization.